# SIADEX HTN ENGINE

####  _______ Begin Installation____________

**Disclaimer** : The installation steps are only needed until up_siadex is published on pypi and the changes on the unified-planning fork are merged into the origin package

In [ ]:
# Make sure the following packages are installed on the system: python-dev libreadline-dev. Those are needed for the execution of Siadex
!apt-get update
!apt-get install -y python-dev libreadline-dev

In [ ]:
# Clonning the repos
!git clone https://github.com/UGR-IntelligentSystemsGroup/unified-planning.git
!git clone https://github.com/UGR-IntelligentSystemsGroup/up-siadex.git

In [ ]:
# Install the packages
# %%capture
%pip install ./unified-planning
%pip install ./up-siadex

####  _______ End Installation____________

## Importing Unified Planning and UP_SIADEX


In [1]:

import unified_planning as up
from unified_planning.shortcuts import *
from unified_planning.model.htn.hierarchical_problem import HierarchicalProblem, Task, Method
from unified_planning.io import PDDLReader
from unified_planning.io import PDDLWriter
from unified_planning.io.hpdl.hpdl_reader import HPDLReader
from unified_planning.io.hpdl.hpdl_writer import HPDLWriter
from unified_planning.engines.results import PlanGenerationResultStatus

from up_siadex import SIADEXEngine

## Registering the engine

In order to use `SIADEX`, we need to register it among the set of planning engines available for the UP library as follows.

In [2]:
env = up.environment.get_env()
env.factory.add_engine('siadex', __name__, "SIADEXEngine")

## Reading a problem from a file (HDDL)

In [3]:
reader = PDDLReader()
problem = reader.parse_problem("./unified-planning/unified_planning/test/pddl/htn-transport/domain.hddl", "./unified-planning/unified_planning/test/pddl/htn-transport/problem.hddl")

## Solving the problem with siadex

In [4]:
with env.factory.OneshotPlanner(name='siadex') as p:
    result = p.solve(problem)
    if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
        print(f'{p.name()} found a valid plan!')
        print(f'The plan is: \n')
        for i,a in enumerate(result.plan.actions):
            print(f"{i}: {a}")
    else:
        print('No plan found!')

NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_env().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_2155/3527194102.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

SIADEX found a valid plan!
The plan is: 

0: drive(truck-0, city-loc-2, city-loc-1)
1: pick-up(truck-0, city-loc-1, package-0, capacity-0, capacity-1)
2: drive(truck-0, city-loc-1, city-loc-0)
3: drop(truck-0, city-loc-0, package-0, capacity-0, capacity-1)
4: drive(truck-0, city-loc-0, city-loc-1)
5: pick-up(truck-0, city-loc-1, package-1, capacity-0, capacity-1)
6: drive(truck-0, city-loc-1, city-loc-0)
7: drop(truck-0, city-loc-0, package-1, capacity-0, capacity-1)


/mnt/e/newDesktop/void/projects/siadex/up-siadex/Notebooks/unified-planning/unified_planning/engines/mixins/oneshot_planner.py:73: UserWarning: <function SIADEXEngine.name at 0x7fab86372320> cannot solve this kind of problem!
  warn(msg)


Lets wrap it inside a function so we don't need to write it again

In [5]:
def solve_with_siadex(problem):
    with env.factory.OneshotPlanner(name='siadex') as p:
        result = p.solve(problem)
        print(result.log_messages)
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print(f'{p.name()} found a valid plan!')
            print(f'The plan is: \n')
            for i,a in enumerate(result.plan.actions):
                print(f"{i}: {a}")
            return result.plan
        else:
            return []
            print('No plan found!')

## Example of a hierarchical  problem done with UPF

In [6]:
htn = HierarchicalProblem()

#_________Objects_________#
Location = UserType("Location")

l1 = htn.add_object("l1", Location)
l2 = htn.add_object("l2", Location)
l3 = htn.add_object("l3", Location)
l4 = htn.add_object("l4", Location)

#_________Fluents_________#
loc = Fluent("is_on", l=Location)
htn.add_fluent(loc, default_initial_value=False)

connected = Fluent("connected", l1=Location, l2=Location)
htn.add_fluent(connected, default_initial_value=False)
htn.set_initial_value(connected(l1, l2), True)
htn.set_initial_value(connected(l2, l3), True)
htn.set_initial_value(connected(l3, l4), True)
htn.set_initial_value(connected(l4, l3), True)
htn.set_initial_value(connected(l3, l2), True)
htn.set_initial_value(connected(l2, l1), True)
htn.set_initial_value(loc(l1), True)


#_________Actions_________#
move = InstantaneousAction("move", l_from=Location, l_to=Location)
l_from = move.parameter("l_from")
l_to = move.parameter("l_to")
move.add_precondition(loc(l_from))
move.add_precondition(connected(l_from, l_to))
move.add_effect(loc(l_from), False)
move.add_effect(loc(l_to), True)
htn.add_action(move)

#_________Task_________#
go = htn.add_task("go", target=Location)

#_________Methods_________#
go_noop = Method("go-noop", target=Location)
go_noop.set_task(go)
target = go_noop.parameter("target")
go_noop.add_precondition(loc(target))

htn.add_method(go_noop)

go_recursive = Method(
    "go-recursive", source=Location, inter=Location, target=Location
)

go_recursive.set_task(go, go_recursive.parameter("target"))

source = go_recursive.parameter("source")
inter = go_recursive.parameter("inter")
target = go_recursive.parameter("target")

go_recursive.add_precondition(loc(source))
go_recursive.add_precondition(connected(source, inter))

t1 = go_recursive.add_subtask(move, source, inter)
t2 = go_recursive.add_subtask(go, target)
go_recursive.set_ordered(t1, t2)
htn.add_method(go_recursive)


#_________Init_________#
go1 = htn.task_network.add_subtask(go, l4)

In [7]:
solve_with_siadex(htn)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 2 of `/tmp/ipykernel_2155/937920758.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

[LogMessage(level=<LogLevel.INFO: 2>, message=''), LogMessage(level=<LogLevel.ERROR: 4>, message='\n')]
SIADEX found a valid plan!
The plan is: 

0: move(l1, l2)
1: move(l2, l3)
2: move(l3, l4)


[move(l1, l2), move(l2, l3), move(l3, l4)]

## Reading a problem from a file (HDPL)

In [8]:
reader = HPDLReader()
miconic = reader.parse_problem("./up-siadex/examples/ipc/Miconic/domain.hpdl", "./up-siadex/examples/ipc/Miconic/problem.hpdl")
rover = reader.parse_problem("./up-siadex/examples/ipc/Rover/domain.hpdl","./up-siadex/examples/ipc/Rover/problem.hpdl")
satellite = reader.parse_problem("./up-siadex/examples/ipc/Satellite/domain.hpdl","./up-siadex/examples/ipc/Satellite/problem.hpdl")
# smartphone = reader.parse_problem("./up-siadex/examples/ipc/SmartPhone/domain.hpdl","./up-siadex/examples/ipc/SmartPhone/problem.hpdl")
transport = reader.parse_problem("./up-siadex/examples/ipc/Transport/domain.hpdl","./up-siadex/examples/ipc/Transport/problem.hpdl")
# translog = reader.parse_problem("./up-siadex/examples/ipc/UM-Translog/domain.hpdl","./up-siadex/examples/ipc/UM-Translog/problem.hpdl")
zeno = reader.parse_problem("./up-siadex/examples/ipc/Zenotravel/domain.hpdl","./up-siadex/examples/ipc/Zenotravel/problem.hpdl")

In [9]:
solve_with_siadex(rover)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 2 of `/tmp/ipykernel_2155/937920758.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

[LogMessage(level=<LogLevel.INFO: 2>, message=''), LogMessage(level=<LogLevel.ERROR: 4>, message='\n\n[Error]: Empty stack\n')]
SIADEX found a valid plan!
The plan is: 



[]

In [10]:
solve_with_siadex(transport)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 2 of `/tmp/ipykernel_2155/937920758.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

[LogMessage(level=<LogLevel.INFO: 2>, message=''), LogMessage(level=<LogLevel.ERROR: 4>, message='\n')]
SIADEX found a valid plan!
The plan is: 

0: drive_primitive(truck_0, city_loc_2, city_loc_1)
1: pick_up_primitive(truck_0, city_loc_1, package_0, capacity_0, capacity_1)
2: drive_primitive(truck_0, city_loc_1, city_loc_0)
3: drop_primitive(truck_0, city_loc_0, package_0, capacity_0, capacity_1)
4: drive_primitive(truck_0, city_loc_0, city_loc_1)
5: pick_up_primitive(truck_0, city_loc_1, package_1, capacity_0, capacity_1)
6: drive_primitive(truck_0, city_loc_1, city_loc_2)
7: drop_primitive(truck_0, city_loc_2, package_1, capacity_0, capacity_1)


[drive_primitive(truck_0, city_loc_2, city_loc_1), pick_up_primitive(truck_0, city_loc_1, package_0, capacity_0, capacity_1), drive_primitive(truck_0, city_loc_1, city_loc_0), drop_primitive(truck_0, city_loc_0, package_0, capacity_0, capacity_1), drive_primitive(truck_0, city_loc_0, city_loc_1), pick_up_primitive(truck_0, city_loc_1, package_1, capacity_0, capacity_1), drive_primitive(truck_0, city_loc_1, city_loc_2), drop_primitive(truck_0, city_loc_2, package_1, capacity_0, capacity_1)]

In [11]:
solve_with_siadex(zeno)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 2 of `/tmp/ipykernel_2155/937920758.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

[LogMessage(level=<LogLevel.INFO: 2>, message=''), LogMessage(level=<LogLevel.ERROR: 4>, message='\n')]
SIADEX found a valid plan!
The plan is: 

0: fly_primitive(a1, c2, c4, f6, f5)
1: board_primitive(p1, a1, c4)
2: fly_primitive(a1, c4, c3, f5, f4)
3: debark_primitive(p1, a1, c3)
4: fly_primitive(a1, c3, c2, f4, f3)
5: board_primitive(p4, a1, c2)
6: fly_primitive(a1, c2, c4, f3, f2)
7: debark_primitive(p4, a1, c4)
8: fly_primitive(a1, c4, c1, f2, f1)
9: board_primitive(p3, a1, c1)
10: fly_primitive(a1, c1, c4, f1, f0)
11: debark_primitive(p3, a1, c4)
12: refuel_primitive(a1, f0, f2)
13: fly_primitive(a1, c4, c1, f2, f0)
14: board_primitive(p2, a1, c1)
15: refuel_primitive(a1, f0, f2)
16: fly_primitive(a1, c1, c4, f2, f0)
17: debark_primitive(p2, a1

[fly_primitive(a1, c2, c4, f6, f5), board_primitive(p1, a1, c4), fly_primitive(a1, c4, c3, f5, f4), debark_primitive(p1, a1, c3), fly_primitive(a1, c3, c2, f4, f3), board_primitive(p4, a1, c2), fly_primitive(a1, c2, c4, f3, f2), debark_primitive(p4, a1, c4), fly_primitive(a1, c4, c1, f2, f1), board_primitive(p3, a1, c1), fly_primitive(a1, c1, c4, f1, f0), debark_primitive(p3, a1, c4), refuel_primitive(a1, f0, f2), fly_primitive(a1, c4, c1, f2, f0), board_primitive(p2, a1, c1), refuel_primitive(a1, f0, f2), fly_primitive(a1, c1, c4, f2, f0), debark_primitive(p2, a1, c4), goal_action_primitive]

In [12]:
solve_with_siadex(miconic)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 2 of `/tmp/ipykernel_2155/937920758.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

[LogMessage(level=<LogLevel.INFO: 2>, message=''), LogMessage(level=<LogLevel.ERROR: 4>, message='\n')]
SIADEX found a valid plan!
The plan is: 

0: move_primitive(f0, f0)
1: board_primitive(p0, f0)
2: move_primitive(f0, f1)
3: debark_primitive(p0, f1)
4: move_primitive(f1, f3)
5: board_primitive(p2, f3)
6: move_primitive(f3, f1)
7: debark_primitive(p2, f1)
8: move_primitive(f1, f2)
9: board_primitive(p4, f2)
10: move_primitive(f2, f1)
11: debark_primitive(p4, f1)
12: move_primitive(f1, f0)
13: board_primitive(p1, f0)
14: move_primitive(f0, f3)
15: debark_primitive(p1, f3)
16: move_primitive(f3, f3)
17: board_primitive(p3, f3)
18: move_primitive(f3, f2)
19: debark_primitive(p3, f2)


[move_primitive(f0, f0), board_primitive(p0, f0), move_primitive(f0, f1), debark_primitive(p0, f1), move_primitive(f1, f3), board_primitive(p2, f3), move_primitive(f3, f1), debark_primitive(p2, f1), move_primitive(f1, f2), board_primitive(p4, f2), move_primitive(f2, f1), debark_primitive(p4, f1), move_primitive(f1, f0), board_primitive(p1, f0), move_primitive(f0, f3), debark_primitive(p1, f3), move_primitive(f3, f3), board_primitive(p3, f3), move_primitive(f3, f2), debark_primitive(p3, f2)]